In [20]:
# --- 1. 라이브러리 및 모듈 임포트 ---

# 데이터 처리 라이브러리
import pandas as pd
import numpy as np

# 머신러닝 모델 및 평가 도구
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

# 시스템 및 경로 관련 라이브러리
import sys
import os
from datetime import datetime
import warnings

# 경고 메시지 무시
warnings.filterwarnings('ignore')

In [21]:
# --- 2. 경로 설정 및 커스텀 로거 임포트 ---

# 현재 노트북 파일의 위치를 기준으로 프로젝트 루트 경로를 설정합니다.
# (예: notebooks 폴더에 있다면 ../../src 가 맞습니다)
try:
    src_path = os.path.abspath(os.path.join(os.getcwd(), "../../src/log"))
    sys.path.insert(0, src_path)
    from logger import Logger
    print("Logger 모듈 로드 성공.")
except ImportError:
    print("[오류] Logger 모듈을 찾을 수 없습니다. src/log/logger.py 경로를 확인해주세요.")
    # Logger가 없어도 코드가 중단되지 않도록 임시 클래스 정의
    class Logger:
        def __init__(self, *args, **kwargs): pass
        def write(self, message, **kwargs): print(message)

Logger 모듈 로드 성공.


In [ ]:
# --- 3. 로거 및 경로 초기화 ---

# 로그 파일이 저장될 경로 설정
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
LOG_DIR         = '../../data/logs/price_prediction_transport_logs'
LOG_FILENAME    = f"price_prediction_transport_{timestamp}.log"
LOG_PATH        = os.path.join(LOG_DIR, LOG_FILENAME)

# Directory 생성
os.makedirs(LOG_DIR, exist_ok=True)

# Logger 인스턴스 생성
logger = Logger(log_path=LOG_PATH)

# 데이터 및 제출 파일 경로 설정
DATA_PATH = '../../data/processed/cleaned_data/test_clean.csv'
SUBMISSION_DIR = '../../data/processed/submissions'
SUBMISSION_FILENAME = 'price_prediction_submission.csv'
SUBMISSION_PATH = os.path.join(SUBMISSION_DIR, SUBMISSION_FILENAME)

logger.write("="*50)
logger.write(">> 아파트 가격 예측 모델링 시작")

2025-07-14 06:26:19 | ==================================================
2025-07-14 06:26:19 | >> 아파트 가격 예측 모델링 시작


In [23]:
# --- 4. 데이터 로드 ---

logger.write(">> 정제된 Train/Test 데이터를 로드합니다...")
try:
    # 학습 데이터와 테스트 데이터를 각각 로드
    train_df_clean = pd.read_csv('../../data/processed/cleaned_data/train_clean.csv')
    test_df_clean = pd.read_csv('../../data/processed/cleaned_data/test_clean.csv')
    
    # 두 데이터를 합치기 전에 isTest 컬럼으로 구분
    train_df_clean['isTest'] = 0
    test_df_clean['isTest'] = 1
    
    # 하나의 데이터프레임으로 합침
    df = pd.concat([train_df_clean, test_df_clean])
    
    logger.write(">> 데이터 로드 및 병합 완료.")
except FileNotFoundError:
    logger.write(f">> [오류] train_clean.csv 또는 test_clean.csv 파일을 찾을 수 없습니다.", print_error=True)
    df = None

2025-07-14 06:26:19 | >> 정제된 Train/Test 데이터를 로드합니다...
2025-07-14 06:26:23 | >> 데이터 로드 및 병합 완료.


In [24]:
# --- 5. 피처 선택 및 데이터 분리 ---

if df is not None:
    try:
        # isTest 컬럼을 사용해 train/test 데이터 분리
        train_df = df[df['isTest'] == 0].copy()
        test_df = df[df['isTest'] == 1].copy()
        logger.write(">> Train/Test 데이터 분리 완료.")

        # 예측에 사용할 피처(X)와 타겟(y) 정의
        features = [col for col in df.columns if col not in ['isTest', 'target']]
        
        X_train_raw = train_df[features]
        y_train = train_df['target']
        X_test_raw = test_df[features]

        # 범주형(object) 피처 자동 탐지
        categorical_features = X_train_raw.select_dtypes(include=['object']).columns.tolist()
        
        logger.write(f">> 예측에 사용할 피처 개수: {len(features)}개")
        logger.write(f">> 범주형 피처: {categorical_features}")
    except Exception as e:
        logger.write(f">> [오류] 피처 선택 및 데이터 분리 중 문제 발생: {e}", print_error=True)

2025-07-14 06:26:24 | >> Train/Test 데이터 분리 완료.
2025-07-14 06:26:24 | >> 예측에 사용할 피처 개수: 28개
2025-07-14 06:26:24 | >> 범주형 피처: ['계약일자', '자치구', '법정동', '브랜드등급', '지하철최단거리', '반경_1km_지하철역_수', '반경_500m_지하철역_수', '반경_300m_지하철역_수', '버스최단거리', '반경_1km_버스정류장_수', '반경_500m_버스정류장_수', '반경_300m_버스정류장_수']


In [25]:
# --- 6. 범주형 피처 인코딩 ---

if 'X_train_raw' in locals():
    try:
        # LabelEncoder를 사용하여 각 범주형 피처를 수치로 변환
        label_encoders = {}
        for col in categorical_features:
            le = LabelEncoder()
            all_categories = pd.concat([X_train_raw[col], X_test_raw[col]]).astype(str).unique()
            le.fit(all_categories)
            
            X_train_raw[col] = le.transform(X_train_raw[col].astype(str))
            X_test_raw[col] = le.transform(X_test_raw[col].astype(str))
            label_encoders[col] = le
        logger.write(">> 범주형 피처 Label Encoding 완료.")
    except Exception as e:
        logger.write(f">> [오류] 범주형 피처 인코딩 중 문제 발생: {e}", print_error=True)

2025-07-14 06:26:25 | >> 범주형 피처 Label Encoding 완료.


In [26]:
# --- 7. 모델 학습 및 평가 ---

if 'X_train_raw' in locals():
    try:
        # 학습 데이터를 다시 train/validation 세트로 분리
        X_train, X_val, y_train_split, y_val_split = train_test_split(
            X_train_raw, y_train, test_size=0.2, random_state=42
        )

        # LightGBM 모델 학습
        logger.write(">> LightGBM 모델 학습을 시작합니다...")
        lgbm = lgb.LGBMRegressor(random_state=42)
        lgbm.fit(X_train, y_train_split)

        # 검증 데이터셋에 대한 예측 및 성능 평가
        y_pred = lgbm.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val_split, y_pred))
        logger.write(f">> 모델 학습 완료. Validation RMSE: {rmse:.4f}")

        # 피처 중요도 확인
        feature_importances = pd.Series(lgbm.feature_importances_, index=features)
        logger.write(">> 상위 10개 피처 중요도:")
        logger.write(str(feature_importances.sort_values(ascending=False).head(10)))
    except Exception as e:
        logger.write(f">> [오류] 모델 학습 및 평가 중 문제 발생: {e}", print_error=True)


2025-07-14 06:26:26 | >> LightGBM 모델 학습을 시작합니다...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.009519 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3111
[LightGBM] [Info] Number of data points in the train set: 878273, number of used features: 28
[LightGBM] [Info] Start training from score 57582.439147
2025-07-14 06:26:27 | >> 모델 학습 완료. Validation RMSE: 12749.2160
2025-07-14 06:26:27 | >> 상위 10개 피처 중요도:
2025-07-14 06:26:27 | 전용면적              511
연식                335
법정동               263
자치구               256
계약일자              178
성비(남/여)           170
지하철최단거리           145
총인구수              141
반경_1km_버스정류장_수    128
아파트이름길이           120
dtype: int32


In [ ]:
# --- 8. 최종 예측 및 제출 파일 생성 ---
if 'X_train_raw' in locals():
    logger.write("\n>> 최종 모델 학습 및 제출 파일 생성을 시작합니다...")
    try:
        # 전체 학습 데이터로 최종 모델 재학습
        final_model = lgb.LGBMRegressor(random_state=42)
        final_model.fit(X_train_raw, y_train)
        
        # 테스트 데이터에 대한 예측 수행
        final_predictions = final_model.predict(X_test_raw)
        
        # 제출 파일 생성
        try:
            sample_submission_df = pd.read_csv(SUBMISSION_PATH)
            sample_submission_df['target'] = final_predictions
            sample_submission_df.to_csv(SUBMISSION_PATH, index=False)
        except FileNotFoundError:
            logger.write(">> sample_submission.csv 파일을 찾을 수 없어 새로 생성합니다.", print_error=True)
            submission_df = pd.DataFrame({'target': final_predictions})
            submission_df.to_csv(SUBMISSION_PATH, index=False)
            
        logger.write(f">> 제출 파일 '{SUBMISSION_PATH}' 생성 완료.")
    except Exception as e:
        logger.write(f">> [오류] 최종 예측 및 제출 파일 생성 중 문제 발생: {e}", print_error=True)
    
    logger.write(">> 모델링 종료")
    logger.write("="*50 + "\n")

2025-07-14 06:26:27 | 
>> 최종 모델 학습 및 제출 파일 생성을 시작합니다...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.016775 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3109
[LightGBM] [Info] Number of data points in the train set: 1097842, number of used features: 28
[LightGBM] [Info] Start training from score 57571.638850
2025-07-14 06:26:28 | >> 제출 파일 '../../data/processed/submissions/test_submission.csv' 생성 완료.
2025-07-14 06:26:28 | >> 모델링 종료
2025-07-14 06:26:28 | ==================================================

